In [3]:
import nltk
import warnings
warnings.filterwarnings('ignore')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\sanjo\AppData\Roaming\nltk_data...


In [7]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time
import pprint

date_sentiments = {}

for i in range(1,11):
    page = urlopen('https://www.businesstimes.com.sg/search/facebook?page='+str(i)).read()
    soup = BeautifulSoup(page, features="html.parser")
    posts = soup.findAll("div", {"class": "media-body"})
    for post in posts:
        time.sleep(1)
        url = post.a['href']
        date = post.time.text
        print(date, url)
        try:
            link_page = urlopen(url).read()
        except:
            pass
            #url = url[:-2]
            #link_page = urlopen(url).read()
        link_soup = BeautifulSoup(link_page)
        sentences = link_soup.findAll("p")
        passage = ""
        for sentence in sentences:
            passage += sentence.text
        sentiment = sia.polarity_scores(passage)['compound']
        date_sentiments.setdefault(date, []).append(sentiment)

date_sentiment = {}

for k,v in date_sentiments.items():
    date_sentiment[datetime.strptime(k, '%d %b %Y').date() + timedelta(days=1)] = round(sum(v)/float(len(v)),3)

earliest_date = min(date_sentiment.keys())

print(date_sentiment)

29 Mar 2019 https://www.businesstimes.com.sg/technology/facebook-bans-white-nationalism-separatism-on-its-platforms
28 Mar 2019 https://www.businesstimes.com.sg/real-estate/us-accuses-facebook-of-discrimination-over-housing-ads
28 Mar 2019 https://www.businesstimes.com.sg/technology/facebook-bans-white-nationalism-white-separatism-on-its-platforms
26 Mar 2019 https://www.businesstimes.com.sg/government-economy/eu-lawmakers-back-copyright-reforms-targeting-google-facebook
25 Mar 2019 https://www.businesstimes.com.sg/technology/facebook-google-face-steeper-privacy-fines-under-australia-plan
22 Mar 2019 https://www.businesstimes.com.sg/technology/amazon-to-launch-mobile-ads-in-threat-to-google-and-facebook
22 Mar 2019 https://www.businesstimes.com.sg/technology/facebook-admits-storing-passwords-in-plain-text
20 Mar 2019 https://www.businesstimes.com.sg/technology/facebook-and-google-arent-the-only-ones-tracking%C2%A0your-clicks
16 Mar 2019 https://www.businesstimes.com.sg/technology/acade

28 Nov 2018 https://www.businesstimes.com.sg/government-economy/facebook-was-warned-of-alleged-russian-meddling-back-in-2014
27 Nov 2018 https://www.businesstimes.com.sg/technology/lawmakers-criticise-facebooks-zuckerberg-for-uk-parliament-no-show
27 Nov 2018 https://www.businesstimes.com.sg/technology/facebook-secrets-in-bikini-app-flap-stir-international-intrigue
27 Nov 2018 https://www.businesstimes.com.sg/technology/alibaba-market-value-tops-facebook-after-latest-public-relations-crisis-chart
26 Nov 2018 https://www.businesstimes.com.sg/technology/do-you-have-a-moral-duty-to-leave-facebook
23 Nov 2018 https://www.businesstimes.com.sg/technology/facebook-to-pay-100m-euros-in-italian-fiscal-accord
22 Nov 2018 https://www.businesstimes.com.sg/consumer/departing-facebook-communications-chief-says-critics-were-targeted
22 Nov 2018 https://www.businesstimes.com.sg/technology/zuckerberg-says-he-is-not-considering-resigning-from-facebook
21 Nov 2018 https://www.businesstimes.com.sg/technol

In [8]:
print(date_sentiment)

{datetime.date(2019, 3, 30): -0.953, datetime.date(2019, 3, 29): -0.812, datetime.date(2019, 3, 27): 0.807, datetime.date(2019, 3, 26): -0.92, datetime.date(2019, 3, 23): 0.905, datetime.date(2019, 3, 21): -0.807, datetime.date(2019, 3, 17): -0.034, datetime.date(2019, 3, 16): 0.008, datetime.date(2019, 3, 15): 0.121, datetime.date(2019, 3, 14): 0.991, datetime.date(2019, 3, 13): 0.89, datetime.date(2019, 3, 9): 0.011, datetime.date(2019, 3, 8): 0.663, datetime.date(2019, 3, 7): 0.188, datetime.date(2019, 3, 3): -0.974, datetime.date(2019, 3, 2): 0.676, datetime.date(2019, 2, 24): -0.693, datetime.date(2019, 2, 23): 0.998, datetime.date(2019, 2, 22): 0.889, datetime.date(2019, 2, 21): 0.979, datetime.date(2019, 2, 20): 0.871, datetime.date(2019, 2, 19): 0.977, datetime.date(2019, 2, 14): 0.989, datetime.date(2019, 2, 13): 0.568, datetime.date(2019, 2, 12): -0.234, datetime.date(2019, 2, 8): 0.387, datetime.date(2019, 2, 3): -0.086, datetime.date(2019, 2, 2): 0.033, datetime.date(2019, 

In [10]:
from __future__ import (absolute_import, division, print_function,
                            unicode_literals)

# %matplotlib inline
# import warnings
# warnings.filterwarnings('ignore')

import backtrader as bt
import backtrader.indicators as btind
import datetime
import os.path
import sys

class Sentiment(bt.Indicator):
    lines = ('sentiment',)
    plotinfo = dict(
        plotymargin=0.15,
        plothlines=[0],
        plotyticks=[1.0, 0, -1.0])
    
    def next(self):
        self.date = self.data.datetime
        date = bt.num2date(self.date[0]).date()
        prev_sentiment = self.sentiment
        if date in date_sentiment:
            self.sentiment = date_sentiment[date]
        self.lines.sentiment[0] = self.sentiment


class SentimentStrat(bt.Strategy):
    params = (
        ('period', 15),
        ('printlog', True),
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function for this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        # Keep track of pending orders
        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.period)
        self.date = self.data.datetime
        self.sentiment = None
        Sentiment(self.data)
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return
        
        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))
                
            self.bar_executed = len(self)     
            
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
            
        # Write down: no pending order
        self.order = None
        
    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
    
    ### Main Strat ###
    def next(self):
        # log closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])
        
        date = bt.num2date(self.date[0]).date()
        prev_sentiment = self.sentiment
        if date in date_sentiment:
            self.sentiment = date_sentiment[date]
        
        # Check if an order is pending. if yes, we cannot send a 2nd one
        if self.order:
            return
        print(self.sentiment)
        # If not in the market and previous sentiment not none
        if not self.position and prev_sentiment:
            # buy if current close more than sma AND sentiment increased by >= 0.5
            if self.dataclose[0] > self.sma[0] and self.sentiment - prev_sentiment >= 0.5:
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                self.order = self.buy()
                
        # Already in the market and previous sentiment not none
        elif prev_sentiment:
            # sell if current close less than sma AND sentiment decreased by >= 0.5
            if self.dataclose[0] < self.sma[0] and self.sentiment - prev_sentiment <= -0.5:
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
                self.order = self.sell()

    def stop(self):
        self.log('(MA Period %2d) Ending Value %.2f' %
                 (self.params.period, self.broker.getvalue()), doprint=True)
        

if __name__ == '__main__':
    cerebro = bt.Cerebro()
    
    # Strategy
    cerebro.addstrategy(SentimentStrat)

    # Data Feed
    data = bt.feeds.YahooFinanceData(
        dataname = 'FB',
        fromdate = earliest_date,
        todate = datetime.datetime(2018,11,25),
        reverse = False
    )
    
    cerebro.adddata(data)

    cerebro.broker.setcash(100000.0)
    cerebro.addsizer(bt.sizers.FixedSize, stake=10)
    cerebro.broker.setcommission(commission=0.001)
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    cerebro.run()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    
    cerebro.plot()

Starting Portfolio Value: 100000.00


TypeError: must be real number, not LineBuffer